In [0]:
#Import Libraries 
import dlt
from pyspark import pipelines as dp
from pyspark.sql.functions import *
from pyspark.sql.types import *
#Creating a Delta Live Table to ingest data from the bronze


In [0]:
dataset_rules = {
    "R1" : "material_id is not null",
    "R2" : "material_name is not null",
    "R3" : "unit_cost > 0"
}

In [0]:
@dlt.table
@dlt.expect_all(dataset_rules)
def bronze_material_master():
    df = spark.readStream \
        .format("delta") \
        .option("readChangeFeed", "true") \
        .table("workspace.materialmasterdetail.material_master")
    return df



In [0]:
@dlt.table
@dlt.expect_all_or_drop(dataset_rules)
def silver_material_master():
    df = spark.readStream.table("LIVE.bronze_material_master")
    df = df.withColumn("safety_stock", col("safety_stock").cast(IntegerType()))
    df = df.withColumn("reorder_level", col("reorder_level").cast(IntegerType()))
    df = df.withColumn("unit_cost", df.unit_cost.cast(DoubleType()))
    df = df.withColumn("last_updated", col("last_updated").cast(DateType()))
    df = df.withColumn("lead_time_days", col("lead_time_days").cast(IntegerType()))
    return df